# 1. Type hinting v Pythonu

Type hinting (anotace typů) umožňuje popsat, jaké typy hodnot funkce očekává a co vrací. Samotný běh programu tím obvykle neomezíme, ale výrazně pomůžeme statické kontrole, nápovědě v IDE a čitelnosti kódu.

V tomto notebooku projdeme:

1. základní syntaxi anotací,
2. základní typy,
3. modul `typing`,
4. type aliasy,
5. generika,
6. type hinty pro NumPy,
7. práci s anotacemi za běhu,
8. odkazy na typ třídy uvnitř její definice.


## 1.1 Základní syntaxe

Pro argumenty a návratovou hodnotu funkce používáme anotace typů. Typ argumentu zapisujeme za název argumentu pomocí `:` a návratový typ za `->`.


In [ ]:
def jmeno_funkce(argument1: int, argument2: str) -> int:
    return 0

Stejně můžeme typy anotovat i u samostatných proměnných.


In [ ]:
cislo_pi : float = 3.14
nejaky_text : str = "Ahoj světe!"

Linter nebo nástroj pro statickou analýzu (např. `mypy`) umí odhalit, že předáváme špatný typ argumentu. Type hinty se ale standardně nevynucují za běhu programu.


In [ ]:
jmeno_funkce(cislo_pi, nejaky_text)

## 1.2 Základní type hinty

Jako type hinty můžeme použít běžné vestavěné typy, například:

- `int`, `float`, `bool`, `str`
- `bytes`, `bytearray`, `memoryview`
- `list`, `tuple`, `dict`, `set`, `frozenset`
- `range`, `complex`, `None`

V praxi často stačí právě tyto základní typy.


In [ ]:
def foo2(bar: list[int]) -> int:
    return 0

promenna: list[int] = [1, 2, 3]
foo2(promenna)


In [ ]:
def foo3(bar: range) -> None:
    for i in bar:
        print(i)

a = range(2)
foo3(a)

Type hinty mohou obsahovat i vlastní třídy.


In [ ]:
class Trida:
    def __init__(self, promenna: int):
        self.promenna = promenna

    def metoda(self, promenna: int) -> int:
        return self.promenna + promenna
    
def foo4(bar: Trida) -> int:
    return bar.metoda(1)

promenna = Trida(1)
foo4(promenna)

Podívejte se na rozdíl v nápovědě k metodě `metoda`: v prvním případě IDE ví, jaké argumenty metoda čeká a co vrací. Bez type hintů je tato informace výrazně slabší.


In [ ]:
def foo5(bar):
    return bar.metoda(1)

foo5(promenna)


## 1.3 Modul `typing`

Modul `typing` rozšiřuje možnosti anotací o specializované typy.

Důležité pro aktuální praxi:

- od Pythonu 3.9 preferujeme zápis vestavěných kolekcí (`list[int]`, `dict[str, int]`),
- aliasy jako `typing.List`, `typing.Dict`, `typing.Tuple` jsou zpětně kompatibilní, ale jsou vedené jako deprecated,
- speciální konstrukce jako `Union`, `Optional`, `Literal`, `Annotated` nebo `TypeVar` se dál běžně používají.


In [ ]:
from typing import Union, Optional

# list s definovaným typem prvků
seznam_celych_cisel: list[int] = [1, 2, 3]

# tuple s definovaným typem a počtem prvků
ntice_celych_cisel: tuple[int, int, int] = (1, 2, 3)

# dict s definovaným typem klíče a hodnoty
slovnik_celych_cisel: dict[str, int] = {"jedna": 1, "dva": 2, "tri": 3}


### 1.3.1 Kombinace typů

`Union` a `Optional` se hodí, když funkce může pracovat s více variantami vstupu nebo návratové hodnoty.


#### 1.3.1.1 Union

`Union` říká, že hodnota může být jedním z více typů.


In [ ]:
from typing import Union


def vypis_hodnotu(hodnota: Union[int, str]) -> None:
    print(f"Hodnota je: {hodnota}")


vypis_hodnotu(42)  # Hodnota je: 42
vypis_hodnotu("Ahoj")  # Hodnota je: Ahoj


#### 1.3.1.2 Optional

`Optional[T]` je zkratka pro `Union[T, None]`, tedy hodnota typu `T` nebo `None`.


In [ ]:
from typing import Optional


def najdi_index(hledany_prvek: str, seznam_prvku: list[str]) -> Optional[int]:
    if hledany_prvek in seznam_prvku:
        return seznam_prvku.index(hledany_prvek)
    return None


prvky = ["jablko", "hruska", "banan"]
index_jablka = najdi_index("jablko", prvky)  # index_jablka = 0
index_pomeranc = najdi_index("pomeranc", prvky)  # index_pomeranc = None


#### 1.3.1.3 Syntaxe od Pythonu 3.10

Od Pythonu 3.10 můžeme místo `Union[int, str]` psát kratší zápis `int | str`.


In [ ]:
# Příklad s Union

def vypis_hodnotu(hodnota: int | str) -> None:
    print(f"Hodnota je: {hodnota}")


vypis_hodnotu(42)  # Hodnota je: 42
vypis_hodnotu("Ahoj")  # Hodnota je: Ahoj

# Příklad s Optional

def najdi_index(hledany_prvek: str, seznam_prvku: list[str]) -> int | None:
    if hledany_prvek in seznam_prvku:
        return seznam_prvku.index(hledany_prvek)
    return None


prvky = ["jablko", "hruska", "banan"]
index_jablka = najdi_index("jablko", prvky)  # index_jablka = 0
index_pomeranc = najdi_index("pomeranc", prvky)  # index_pomeranc = None


### 1.3.2 Konkrétní hodnoty pomocí `Literal`

`Literal` se hodí, když je povolená jen omezená množina konkrétních hodnot.


In [ ]:
from typing import Literal


def set_status(status: Literal['pending', 'approved', 'rejected']) -> None:
    print(f"Setting status to: {status}")


# This will work
set_status('approved')

# This will raise a type error during static type checking
set_status('unknown')


### 1.3.3 Dodatečné informace přes `Annotated`

`Annotated` umožňuje přidat k typu metadata pro nástroje nebo dokumentaci.

Důležité: samotný Python tato metadata nevynucuje. Pokud je nástroj neinterpretuje, chová se typ jako původní `T`.


In [ ]:
from dataclasses import dataclass
from typing import Annotated


@dataclass(frozen=True)
class MaxLen:
    value: int


OmezenaJmena = Annotated[list[str], MaxLen(5)]


def process_names(names: OmezenaJmena) -> None:
    for name in names:
        print(f"Processing name: {name}")


names = ["Alice", "Bob", "Charlie", "David", "Eve"]
process_names(names)

names2 = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank"]
process_names(names2)  # Bez runtime chyby; metadata je určeno hlavně pro nástroje.


## 1.4 Type aliasy

Type alias zkracuje zápis složitějšího typu a zlepšuje čitelnost.

Od Pythonu 3.12 lze alias zapisovat i pomocí `type` statement, ale kvůli kompatibilitě se staršími verzemi se často používá i prosté přiřazení.


In [ ]:
# Python 3.12+ alternativa:
# type Souradnice = tuple[float, float]
# type Cesta = list[Souradnice]

Souradnice = tuple[float, float]
Cesta = list[Souradnice]


def vypocet_vzdalenosti(cesta: Cesta) -> float:
    vzdalenost = 0.0
    for i, souradnice in enumerate(cesta):
        if i == 0:
            continue
        x1, y1 = cesta[i - 1]
        x2, y2 = souradnice
        vzdalenost += ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
    return vzdalenost


cesta = [(0.1, 0.1), (1.1, 1.1), (2.1, 2.1)]
vypocet_vzdalenosti(cesta)


## 1.5 Generika

Generika použijeme ve chvíli, kdy chceme zachovat informaci, že několik hodnot má stejný typ, ale tento typ nemusíme předem pevně určit.


In [ ]:
from typing import TypeVar

T = TypeVar("T")


def najdi_prvni(prvky: list[T], cil: T) -> int:
    for i, prvek in enumerate(prvky):
        if prvek == cil:
            return i
    return -1


prvky = [1, 2, 3]
najdi_prvni(prvky, 2)

prvky = ["jablko", "hruska", "banan"]
najdi_prvni(prvky, "hruska")


In [ ]:
prvky = [1.3, 2.4, 3.5]
najdi_prvni(prvky, "3")  # Statická kontrola nahlásí chybný typ druhého argumentu.


`TypeVar` můžeme i omezit na konkrétní povolené typy.


In [ ]:
from typing import TypeVar

OmezenyTyp = TypeVar("OmezenyTyp", int, str)


def najdi_prvni(prvky: list[OmezenyTyp], cil: OmezenyTyp) -> int:
    for i, prvek in enumerate(prvky):
        if prvek == cil:
            return i
    return -1


prvky = [1, 2, 3]
najdi_prvni(prvky, 2)

prvky = ["jablko", "hruska", "banan"]
najdi_prvni(prvky, "hruska")


In [ ]:
prvky = [1.3, 2.4, 3.5]
najdi_prvni(prvky, 3.5)  # Omezený TypeVar zde hlásí nekompatibilní typ.


## 1.6 Type hinty pro NumPy

U polí `ndarray` často potřebujeme popsat nejen to, že jde o pole, ale i datový typ jeho prvků. S tím pomáhá `numpy.typing`.


In [ ]:
import numpy as np
from numpy.typing import NDArray


def process_array(arr: NDArray[np.float64]) -> None:
    print(arr)


input_array: NDArray[np.float64] = np.array([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]
], dtype=np.float64)  # Explicitně nastavíme dtype na float64

process_array(input_array)

In [ ]:
input_array = np.array([1, 2, 3], dtype=np.int8)
process_array(input_array)  # Statický analyzátor upozorní na odlišný dtype.


Typování rozměrů pole (shape) se v NumPy postupně zlepšuje, ale stále nepokrývá všechny scénáře stejně přesně jako typování `dtype`.


## 1.7 Type hinty při běhu

Pro rychlý náhled můžeme použít `__annotations__`, ale obecně je vhodnější `inspect.get_annotations()` (v Pythonu 3.14+ lze použít i `annotationlib.get_annotations()`).


In [ ]:
import inspect


def foo(par: int, par2: str) -> None:
    print(par, par2)


print(foo.__annotations__)
print(inspect.get_annotations(foo))


## 1.8 Odkaz na typ třídy uvnitř její definice

V novějších verzích Pythonu (3.14+) už často nejsou u forward referencí potřeba uvozovky. Pro kompatibilitu se starším kódem je ale stále běžně uvidíte.


In [ ]:
class LepsiCislo:
    def __init__(self, hodnota: int | float) -> None:
        self.hodnota = hodnota

    def __add__(self, other: "LepsiCislo") -> "LepsiCislo":
        return LepsiCislo(self.hodnota + other.hodnota)